In [2]:
### Author: Md Shadman Sakib 5/3/2024; Original Code: Samuel Daramola; CoRAL Lab, Virginia Tech
### information: https://api.tidesandcurrents.noaa.gov/api/prod/#products
### 6 min data can be downloaded for 30 days at a stretch, so a sperate function has to be written to recurrently download a year worth of data.
### This function has not been updated yet
### function has been developed for observed data and prediction data seperately
### A excel containing the required fill will be imported as feeder data into the program. Datum, product description should be changed in that excel
### A post-processing code will merge the data into useable format for data analysis. This can be later connected/updated to directly connect the ranking program.
### Skipping error text for stations where data is not available.

### input: A excel with the station information of NOAA; Standard input file format is = "G:\My Drive\Deflt3D FM Codes - Vtech\DataDownloadAPIs\inputfiles\NOAAStationInfoStormStandardInFile.xlsx"
### output: An excel for each station that was listed in the input excel. The name of each of the excel is as per the code assigned to it

import pandas as pd
from noaa_coops import Station
import os

#-------------------------------FUNCTION START--------------------------------------------------------------#
def fetch_observed(station_id, begin_date, end_date, product, datum):
    station = Station(id=station_id)
    df_observed = station.get_data(
        begin_date=begin_date,
        end_date=end_date,
        product=product,
        datum=datum,
        units="metric",
        time_zone="gmt") 
    return df_observed


def fetch_prediction(station_id, begin_date, end_date, datum):
    station = Station(id=station_id)
    df_prediction = station.get_data(
        begin_date=begin_date,
        end_date=end_date,
        product="predictions",
        datum=datum,
        units="metric",
        time_zone="gmt")
    return df_prediction

#-------------------------------FUNCTION END--------------------------------------------------------------#
#---------------------------------------------------------------------------------------------------------#
#-------------------------------INPUT START---------------------------------------------------------------#
### input file has specific format. For example visit: "G:\My Drive\Deflt3D FM Codes - Vtech\DataDownloadAPIs\inputfiles\NOAAStationInfoStormStandardInFile.xlsx"
station_info=pd.read_excel(r"G:\My Drive\Deflt3D FM Codes - Vtech\DataDownloadAPIs\NOAA\inputfiles\NOAAStationInfoISandy2012 - Copy.xlsx").astype(str) #converts the dataframe into strings
folder_path=r'G:\My Drive\Deflt3D FM Codes - Vtech\DataDownloadAPIs\NOAA\outputfiles\2012Full6min'

#-------------------------------INPUT END-----------------------------------------------------------------#
#---------------------------------------------------------------------------------------------------------#
#-------------------------------DATA PROCESSING START-----------------------------------------------------#

for i in range(len(station_info)):
    station_id=station_info['station'][i][-7:] # extracting the station id
    begin_date=station_info['begin_date'][i]
    end_date=station_info['end_date'][i]
    product=station_info['product'][i]
    datum=station_info['datum'][i]


    filename=station_info['ID'][i]+'.csv'

    print(station_id, begin_date, end_date, product, datum)
    try:
        df_observed=fetch_observed(station_id, begin_date, end_date, product, datum)
        df_prediction=fetch_prediction(station_id, begin_date, end_date, datum)
        #print(df_observed)
        print('###############################')
    except Exception as e:
        print("ERROR: Data not available for this interval at Station: " + station_info['station'][i])
        print('###############################')
        continue 
    
    # dataframe merging
    merged_data = pd.merge(df_observed, df_prediction, left_index=True, right_index=True, suffixes=('_observed (m)', '_predictions (m)'))
    # Convert index to string and then split into 'Date' and 'Time' columns
    merged_data['Date'], merged_data['Time'] = zip(*merged_data.index.astype(str).str.split(' '))
    merged_data = merged_data.reset_index(drop=True)
    
    # Reorder columns and rename headers, multi index to single index
    cols = ['Date', 'Time', 'v_predictions (m)', 'v_observed (m)']
    merged_data = merged_data[cols]
    merged_data.columns = ['Date', 'Time (GMT)', 'Predictions (m)', 'Waterlevel (m)']

    # merging DateTime column into a single column
    merged_data['DateTime (GMT)'] = pd.to_datetime(merged_data['Date'] + ' ' + merged_data['Time (GMT)'])
    merged_data = merged_data.drop(['Date', 'Time (GMT)'], axis=1)
    merged_data = merged_data[['DateTime (GMT)', 'Predictions (m)', 'Waterlevel (m)']]
    merged_data['Datum_Info']= datum


    # writing
    file_path = os.path.join(folder_path, filename)
    merged_data.to_csv(file_path, index=False)




8638610 20120101 20130101 water_level NAVD
###############################


In [2]:
### Author: Md Shadman Sakib 5/3/2024; Original Code: Samual Daramola; CoRAL Lab, Virginia Tech
### input: The path were all the input csv are located.
### output: creates one excel with multiple sheets each representing a different station. This is the input excel for the calibration file.

import pandas as pd
import os

#-------------------------------INPUT START---------------------------------------------------------------#
folder_path = r'G:\My Drive\Deflt3D FM Codes - Vtech\DataDownloadAPIs\NOAA\outputfiles\Isabel2003_6min'
output_file = 'combined_excel_isabel_01082003_01102003.xlsx'
#-------------------------------INPUT END---------------------------------------------------------------#
#-------------------------------DATA PROCESSING START-----------------------------------------------------#

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            # Write the data to a new sheet in the combined file
            # Sheet name is set as the filename without the extension
            df.to_excel(writer, sheet_name=os.path.splitext(filename)[0], index=False)
            
print('CSV files have been combined into one Excel file.')


CSV files have been combined into one Excel file.


,DateTime (GMT),Predictions (m)
0,2012-10-15 00:00:00,0.460660
1,2012-10-15 00:06:00,0.447518
2,2012-10-15 00:12:00,0.433062
3,2012-10-15 00:18:00,0.417370
4,2012-10-15 00:24:00,0.400485
...,...,...
7436,2012-11-14 23:36:00,0.417864
7437,2012-11-14 23:42:00,0.428600
7438,2012-11-14 23:48:00,0.437891
7439,2012-11-14 23:54:00,0.445742
